In [1]:
import matplotlib.pyplot as plt
import csv
from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk
from nltk.corpus import stopwords as stop_words
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import sys
import os
import requests
sys.path.append("../")
from src import puhti_files
from src.common_basis import *

ModuleNotFoundError: No module named 'hereutil'

In [ ]:
SEED = 42
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = puhti_files.genre_data_to_pandas(data="dev", add_labels=True, merge_ecco=True)
df = df[~df['publication_year'].isna()]
df['decade'] = df['publication_year'].apply(lambda x: int(str(int(x))[:-1]+'0'))

In [ ]:
nltk.download('stopwords')

In [ ]:
df['work_id'].unique().tolist()

In [ ]:
df[df['work_id']=='74-hamlet']

In [ ]:
#Topic classifications, combining both sebastian and Eli analisis.
# NA - > The topic only contains weird tokens, doesn't represent anything
topics_names = {1720:{1:'Religion',2:'Goverment/Royal',3:'Romance',4:'Prayer/Bible',5:'Social',6:'Law',7:'Science',8:'Kings',9:'Verbs?/War?',10:'War?'},
                1730:{1:'Religion',2:'Goverment/Kingdom',3:'Religion',4:'Social Concepts',5:'State',6:'Romance',7:'Nature',8:'Locations/People',9:'Kings',10:'Bigger Concepts'},
                1740:{1:'Religion',2:'Romance',3:'Religion',4:'Philosophy',5:'War/Kingdom',6:'Romance',7:'Nature',8:'Law',9:'Royalty',10:'NA?'},
                1750:{1:'Bible/Religion',2:'Romance/Religion',3:'Religion',4:'Nature',5:'Kingdom',6:'Mankind',7:'Kingdom',8:'Country/Issues',9:'NA',10:'NA'},
                1760:{1:'Bible/Religion',2:'State Affairs',3:'Religion',4:'Kingdom',5:'Love/Family',6:'Government affairs',7:'Love/Religious Concepts',8:'Nature',9:'Philosophy',10:'NA'},
                1770:{1:'Kingdom/War',2:'Law/Court',3:'Love/Religion',4:'Higher Power',5:'Nature',6:'Mankind',7:'Kingdom',8:'Love',9:'Church/Religion',10:'NA'},
                1780:{1:'Prayer/Bible',2:'War',3:'Romance',4:'Mandkind',5:'Religion',6:'Law',7:'Trade/Economy',8:'Court/Kingdom',9:'Medicine',10:'NA'},
                1790:{1:'Love',2:'Literature?',3:'Law',4:'Religion',5:'Parliament',6:'Mankind',7:'Romance',8:'Ingredients?',9:'Countries/Land/Economy',10:'NA'}}

In [ ]:
topics_df = pd.DataFrame.from_dict(topics_names)
topics_df

In [ ]:
#df = pd.read_table('/scratch/project_2007227/genre_data/ecco_metadata.tsv')
#df = df[~df['publication_year'].isna()]
#df['decade'] = df['publication_year'].apply(lambda x: int(str(int(x))[:-1]+'0'))

In [ ]:
#Original visualization, just for one book. It must receive an ID that is a str
def book_pattern_vis(id, chunksize=512, ybottom=0):
    #We need to identify the age of the book to use the correct model. Extract year from metadata
    id= str(id)
    year = 1720
    try:
        year = df[df['document_id'] ==id]['publication_year'].values[0]
    except:
        print('Doc id not found')
    year = int(year)
    work_id = df[df['document_id'] ==id]['work_id'].values[0]
    #if the book is from earlier era than 1720, gives erros since we don't have model from that era.
    if year < 1720:
        print('Publication year of the document is not included in the models, please try another document.')
        return True
    
    #if the book is ok, we need to first preprocess the document for the model
    
    #First divide the text into chunks
    with open('/scratch/project_2007227/genre_data/ecco_source/'+str(id)+'.txt', 'r') as f:
        lines = f.readlines()
    
    num_lines = len(lines)
    text = " ".join(lines)
    new_text = '\n'.join([text[i:i+chunksize] for i in range(0, len(text), chunksize)])
    
    with open('/scratch/project_2007227/genre_data/chunk512/'+str(id)+'512.txt', 'w') as f:
        f.write(new_text)
              
    text_file = '/scratch/project_2007227/genre_data/chunk512/'+str(id)+'512.txt'
    documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()[0:num_lines]]

    stopwords = list(stop_words.words("english"))

    sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
    preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()
    
    
    #tp = TopicModelDataPreparation("paraphrase-multilingual-mpnet-base-v2")
    #processed_data= tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
    
    #Nextly we fetch the model that corresponds to the publication year
    modelnumber = ((year // 10) % 10)-2
    decade = int(str(int(year))[:-1]+'0')
    with open('/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/tp_models/tp_model'+str(modelnumber)+'.pickle', 'rb') as tpmodel:
        tpmodeli = pickle.load(tpmodel)

    ctm = ZeroShotTM(bow_size=len(tpmodeli.vocab), contextual_size=768, n_components=10, num_epochs=20)
    ctm.load("/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/ctm_models"+str(modelnumber)+"/contextualized_topic_model_nc_10_tpm_0.0_tpv_0.9_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                          epoch=19)
    processed_data = tpmodeli.transform(preprocessed_documents)
    pred_model = ctm.get_thetas(processed_data, n_samples=20) 
    
    plt.figure(figsize=(10,8))
    plt.plot(pred_model, '.')
    
    plt.ylim((ybottom,1))
    plt.title('Sequential genre prediction for document '+str(id)+' in with 512 token chunks. The document is from the decade '+str(decade) + '. With work id ' + str(work_id))
    plt.legend(topics_names[decade].values(),loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
book_pattern_vis('1299000200')
#Currently the method doesn't support ids starting with 0, to address this feel free to modify str<->int in the code, 

In [ ]:
#Method that given a decade, will visualize 6 samples of it.
#If random_state is True, it will always use the same random state seed, which means that it will always return the same samples
#To see different documents, use random_state=False, so it shows a completly random sample of 6 documents
def book_pattern_vis_sample(decade, chunksize=512, ybottom=0,num_samples=6,random_state=True):
    #We need to identify the age of the book to use the correct model. Extract year from metadata
    if random_state:
        samples = df[df['decade']==decade].sample(n = num_samples,random_state=423)
    else:
        samples = df[df['decade']==decade].sample(n = num_samples)

    ids = samples['document_id'].tolist()
    works_ids = samples['work_id'].tolist()
    stopwords = list(stop_words.words("english"))
    
    modelnumber = ((decade // 10) % 10)-2
    #decade = int(str(int(year))[:-1]+'0')
    with open('/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/tp_models/tp_model'+str(modelnumber)+'.pickle', 'rb') as tpmodel:
        tpmodeli = pickle.load(tpmodel)

    ctm = ZeroShotTM(bow_size=len(tpmodeli.vocab), contextual_size=768, n_components=10, num_epochs=20)
    ctm.load("/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/ctm_models"+str(modelnumber)+"/contextualized_topic_model_nc_10_tpm_0.0_tpv_0.9_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                          epoch=19)
    
    #First divide the text into chunks
    documents = []
    for i in ids:
        len_id = len(str(i))
        if len_id<10:
            i = '0'*(10 - len_id) + str(i)
        else:
            i = str(i)
        with open('/scratch/project_2007227/genre_data/ecco_source/'+str(i)+'.txt', 'r') as f:
            lines = f.readlines()
            num_lines = len(lines)
            text = " ".join(lines)
            new_text = '\n'.join([text[i:i+chunksize] for i in range(0, len(text), chunksize)])
        with open('/scratch/project_2007227/genre_data/chunk512/'+str(i)+'512.txt', 'w') as f:
            f.write(new_text)
        text_file = '/scratch/project_2007227/genre_data/chunk512/'+str(i)+'512.txt'
        documents.append([line.strip() for line in open(text_file, encoding="utf-8").readlines()[0:num_lines]])
    thetas = []
    for i in range(len(documents)):
        sp = WhiteSpacePreprocessingStopwords(documents[i], stopwords_list=stopwords)
        preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()
        processed_data = tpmodeli.transform(preprocessed_documents)
        pred_model = ctm.get_thetas(processed_data, n_samples=20) 
        thetas.append(pred_model)
    
    #tp = TopicModelDataPreparation("paraphrase-multilingual-mpnet-base-v2")
    #processed_data= tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
    
    #Nextly we fetch the model that corresponds to the publication year

    figure, axis = plt.subplots(2, 3, figsize=(16,12),squeeze=False)
    for i in range(len(thetas)):
        axis[i%2, i%3].plot(thetas[i], '.')
        axis[i%2, i%3].axis(ymin=ybottom,ymax=1)
        axis[i%2, i%3].set_title(f"D {ids[i]} - work id {str(works_ids[i])}")
        
    figure.legend(topics_names[decade].values(),loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
book_pattern_vis_sample(1720,random_state=True)

In [ ]:
#Method that uses pre_loaded test datasets to visualize. The problem is that it can only use the chunks, 
#but it doesn't have the original document
def test_loaded_sets(decade):
    modelnumber = ((decade // 10) % 10)-2
    test_sets = []
    with open('/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/tp_models/tp_model'+str(modelnumber)+'.pickle', 'rb') as tpmodel:
        tpmodeli = pickle.load(tpmodel)

    ctm = ZeroShotTM(bow_size=len(tpmodeli.vocab), contextual_size=768, n_components=10, num_epochs=20)
    ctm.load("/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/ctm_models"+str(modelnumber)+"/contextualized_topic_model_nc_10_tpm_0.0_tpv_0.9_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                          epoch=19)

    with open('/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/Test_sets/test_set'+str(modelnumber)+'.pickle', 'rb') as dataset:
        dataseti = pickle.load(dataset)
        test_sets.append(dataseti)

    thetas = []
    for i in test_sets:
        pred_model = ctm.get_thetas(i, n_samples=20)
        thetas.append(pred_model)

    thetas = np.array(thetas)
    thetas_ov = np.mean(thetas,axis=1)
    thetas_ov = np.squeeze(thetas)
    plt.figure(figsize=(10,8))
    plt.plot(thetas_ov, '.')
    plt.ylim((0,1))
    plt.title('Topic prediction for test set with  '+str(thetas_ov.shape[0])+' chunks. The documents are from the decade '+str(decade))
    plt.legend(topics_names[decade].values(),loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
test_loaded_sets(1760)

In [ ]:
test_loaded_sets(1780)

In [ ]:
test_loaded_sets(1720)

In [ ]:
#Method that will sample 6 documents from the given decade and genre.
#The decade must be between 1720 and 1790, and the genre must be the name of the genre, not the number.
#If random_state is True, it will always use the same random state seed, which means that it will always return the same samples
#To see different documents, use random_state=False, so it shows a completly random sample of 6 documents
def sample_per_decade_and_genre(decade,genre, chunksize=512, ybottom=0,num_samples=6,random_state=True):
    #We need to identify the age of the book to use the correct model. Extract year from metadata
    if random_state:
        filt = df[(df['decade']==decade) & (df['main_category_label']==genre)]
        samples = filt.sample(n = min(num_samples,filt.shape[0]),random_state=423)
    else:
        filt = df[(df['decade']==decade) & (df['main_category_label']==genre)]
        samples = filt.sample(n = min(num_samples,filt.shape[0]))

    ids = samples['document_id'].tolist()
    works_ids = samples['work_id'].tolist()
    stopwords = list(stop_words.words("english"))
    
    modelnumber = ((decade // 10) % 10)-2
    #decade = int(str(int(year))[:-1]+'0')
    with open('/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/tp_models/tp_model'+str(modelnumber)+'.pickle', 'rb') as tpmodel:
        tpmodeli = pickle.load(tpmodel)

    ctm = ZeroShotTM(bow_size=len(tpmodeli.vocab), contextual_size=768, n_components=10, num_epochs=20)
    ctm.load("/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/ctm_models"+str(modelnumber)+"/contextualized_topic_model_nc_10_tpm_0.0_tpv_0.9_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                          epoch=19)
    
    #First divide the text into chunks
    documents = []
    for i in ids:
        len_id = len(str(i))
        if len_id<10:
            i = '0'*(10 - len_id) + str(i)
        else:
            i = str(i)
        with open('/scratch/project_2007227/genre_data/ecco_source/'+str(i)+'.txt', 'r') as f:
            lines = f.readlines()
            num_lines = len(lines)
            text = " ".join(lines)
            new_text = '\n'.join([text[i:i+chunksize] for i in range(0, len(text), chunksize)])
        with open('/scratch/project_2007227/genre_data/chunk512/'+str(i)+'512.txt', 'w') as f:
            f.write(new_text)
        text_file = '/scratch/project_2007227/genre_data/chunk512/'+str(i)+'512.txt'
        documents.append([line.strip() for line in open(text_file, encoding="utf-8").readlines()[0:num_lines]])
    thetas = []
    for i in range(len(documents)):
        sp = WhiteSpacePreprocessingStopwords(documents[i], stopwords_list=stopwords)
        preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()
        processed_data = tpmodeli.transform(preprocessed_documents)
        pred_model = ctm.get_thetas(processed_data, n_samples=20) 
        thetas.append(pred_model)
    
    #tp = TopicModelDataPreparation("paraphrase-multilingual-mpnet-base-v2")
    #processed_data= tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
    
    #Nextly we fetch the model that corresponds to the publication year

    figure, axis = plt.subplots(2, 3, figsize=(16,12),squeeze=False)
    for i in range(len(thetas)):
        axis[i%2, i%3].plot(thetas[i], '.')
        axis[i%2, i%3].axis(ymin=ybottom,ymax=1)
        axis[i%2, i%3].set_title(f"D {ids[i]} - work id {str(works_ids[i])[:100]}")
        
    figure.legend(topics_names[decade].values(),loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
#Here are the different labels that can be used in the following method
df['main_category_label'].unique()

In [ ]:
#Example of use
sample_per_decade_and_genre(1790,'Law')

In [ ]:
sample_per_decade_and_genre(1760,'Politics')

In [ ]:
sample_per_decade_and_genre(1740,'Philosophy')

In [ ]:
sample_per_decade_and_genre(1750,'Literature')

In [ ]:
#Method that will sample 6 documents from the given decade and genre.
#The decade must be between 1720 and 1790, and the genre must be the name of the genre, not the number.
#If random_state is True, it will always use the same random state seed, which means that it will always return the same samples
#To see different documents, use random_state=False, so it shows a completly random sample of 6 documents
def sample_per_genre(genre, chunksize=512, ybottom=0,random_state=True):
    #We need to identify the age of the book to use the correct model. Extract year from metadata
    sampled_df = pd.DataFrame()
    for i in range(1720,1800,10):
        if random_state:
            filt = df[(df['decade']==i) & (df['main_category_label']==genre)]
            samples = filt.sample(n = 3,random_state=423)
            sampled_df = pd.concat([sampled_df,samples])
        else:
            filt = df[(df['decade']==i) & (df['main_category_label']==genre)]
            samples = filt.sample(n = 3)
            sampled_df = pd.concat([sampled_df,samples])
    tp_models = []
    ctm_models = []
    stopwords = list(stop_words.words("english"))
    thetas1 = []
    for i in range(1720,1800,10):
        modelnumber = ((i // 10) % 10)-2
        #decade = int(str(int(year))[:-1]+'0')
        with open('/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/tp_models/tp_model'+str(modelnumber)+'.pickle', 'rb') as tpmodel:
            tpmodeli = pickle.load(tpmodel)
        tp_models.append(tpmodeli)
        ctm = ZeroShotTM(bow_size=len(tpmodeli.vocab), contextual_size=768, n_components=10, num_epochs=20)
        ctm.load("/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/ctm_models"+str(modelnumber)+"/contextualized_topic_model_nc_10_tpm_0.0_tpv_0.9_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                              epoch=19)
        ctm_models.append(ctm)
    
        #First divide the text into chunks
        ids = sampled_df[sampled_df['decade']==i]['document_id'].tolist()
        works_ids = sampled_df[sampled_df['decade']==i]['work_id'].tolist()
        documents = []
        for i in ids:
            len_id = len(str(i))
            if len_id<10:
                i = '0'*(10 - len_id) + str(i)
            else:
                i = str(i)
            with open('/scratch/project_2007227/genre_data/ecco_source/'+str(i)+'.txt', 'r') as f:
                lines = f.readlines()
                num_lines = len(lines)
                text = " ".join(lines)
                new_text = '\n'.join([text[i:i+chunksize] for i in range(0, len(text), chunksize)])
            with open('/scratch/project_2007227/genre_data/chunk512/'+str(i)+'512.txt', 'w') as f:
                f.write(new_text)
            text_file = '/scratch/project_2007227/genre_data/chunk512/'+str(i)+'512.txt'
            documents.append([line.strip() for line in open(text_file, encoding="utf-8").readlines()[0:num_lines]])
        thetas = []
        for i in range(len(documents)):
            sp = WhiteSpacePreprocessingStopwords(documents[i], stopwords_list=stopwords)
            preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()
            processed_data = tpmodeli.transform(preprocessed_documents)
            pred_model = ctm.get_thetas(processed_data, n_samples=20) 
            thetas.append(pred_model)
        thetas1.append(thetas)
    
    #tp = TopicModelDataPreparation("paraphrase-multilingual-mpnet-base-v2")
    #processed_data= tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
    
    #Nextly we fetch the model that corresponds to the publication year
    
    figure, axis = plt.subplots(8, 3, figsize=(20,16),squeeze=False)
    for i in range(len(thetas1)):
        for j in range(len(thetas1[i])):
            decade = 1720 + i*10
            axis[i, j].plot(thetas1[i][j], '.')
            axis[i, j].axis(ymin=ybottom,ymax=1)
            axis[i, j].set_title(f"Decade {decade}")
            if j == 2:
                axis[i, j].legend(topics_names[decade].values(),loc='center left', bbox_to_anchor=(1, 0.5))
    
    
    plt.show()

In [ ]:
sample_per_genre('Law')